In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
#add_model.add(Flatten())
add_model.add(Dense(4096, activation='relu'))
add_model.add(Dense(4096, activation='relu'))
add_model.add(Dense(11, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.summary()

batch_size = 8

epochs = 5
nb_train_samples = 29700
nb_validation_samples =3300
# dimensions of our images.
img_width, img_height = 224, 224

# prepare data augmentation configuration
data_generator = ImageDataGenerator(rescale=1. / 255 , validation_split=0.1)


train_data_dir = '/home/serevr/Desktop/NASNET/Class11(224)'


train_generator = data_generator.flow_from_directory(
   train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size, shuffle=True, seed=13,subset="training",
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size, shuffle=False, seed=13,subset="validation",
    class_mode='categorical')


#Y_pred = model.predict_generator(validation_generator, nb_validation_samples//batch_size + 1)
#y_pred = np.argmax(Y_pred, axis=1)

#top_values, top_indices = K.get_session().run(tf.nn.top_k(y_pred, k= 5))

# compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


# fine-tune the model
history = model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    #samples_per_epoch=3712,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=412
    #nb_val_samples=nb_validation_samples
    )

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/home/serevr/anaconda3/envs/en1/lib/python3.5/site-packages/ipykernel_launcher.py:75: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/serevr/anaconda3/envs/en1/lib/python3.5/site-packages/ipykernel_launcher.py:75: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=3712, validation_steps=412, epochs=5)`


Epoch 1/5
2014/3712 [===============>..............] - ETA: 4:48 - loss: 0.4955 - acc: 0.8296

KeyboardInterrupt: 